In [1]:
# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
# Vector stores
import fitz  # PyMuPDF 
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter, MarkdownHeaderTextSplitter, MarkdownTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader ,UnstructuredPDFLoader
from langchain_community.retrievers import BM25Retriever, KNNRetriever 
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever, OktBM25Retriever
from langchain.docstore.document import Document
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
import pymupdf4llm
import time
import re
from konlpy.tag import Okt
from pdf2image import convert_from_path
import pytesseract
from konlpy.tag import Kkma
# etc
import os
import pandas as pd
from tqdm import tqdm
import unicodedata
import logging
from PyPDF2 import PdfReader
import json
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
print(device)

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# intfloat/multilingual-e5-small
# jhgan/ko-sroberta-multitask

def get_embedding():
    
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        
        encode_kwargs={'normalize_embeddings': True})
    return embeddings
def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", " ").replace("", " ")  # 잘못된 인코딩 문자 제거
    return text

def format_docs(docs):
    context = ""
    
    for doc in docs:
        # Header 정보를 순서대로 추가
        for header_level in range(1, 6):
            header_key = f'Header{header_level}'
            if header_key in doc.metadata:
                context += f"{header_key}: {doc.metadata[header_key]}\n"
        # 문서 내용 추가
        context += doc.page_content
        context += '\n---\n'
    return context


In [3]:
def process_pdf(pdf_path): 
    md_text = pymupdf4llm.to_markdown(pdf_path)

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=True)
    splits = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=700, chunk_overlap=60
    )

    splits = text_splitter.split_documents(splits)

    for i in splits:
        metadata = {'Source_path': pdf_path}
        i.metadata = {**i.metadata, **metadata}
    return splits


def make_db(df):
    documents = []
    
    pdf_files = df['Source_path'].unique()
    for pdf_file in tqdm(pdf_files):
        # 문서 로드
        documents.extend(process_pdf(pdf_file))
        
    print(f"Total number of documents: {len(documents)}")

    faiss = FAISS.from_documents(documents, embedding=get_embedding())
    return faiss

In [4]:
def fewshot_db(df):
    df = df.drop('SAMPLE_ID', axis=1)
    df = df.drop('Source_path', axis=1)
    df = df.to_dict(orient='records')
    print("Loaded Fewshot Set:", len(df))
    to_vectorize = ["\n\n".join(normalize_string(value) for value in example.values()) for example in df]
    faiss = FAISS.from_texts(to_vectorize, embedding=get_embedding())
    # bm = BM25Retriever.from_texts(to_vectorize)
    # knn = KNNRetriever.from_texts(to_vectorize, embeddings=get_embedding())
    return faiss

In [5]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

In [6]:
def format_docs(docs):
    context = ""
    for doc in docs:
        # Header 정보를 순서대로 추가
        for header_level in range(1, 6):
            header_key = f'Header {header_level}'
            if header_key in doc.metadata:
                context += f"{header_key}: {doc.metadata[header_key]}\n"
        
        # 문서 내용 추가
        context += doc.page_content
        context += '\n---\n'
    return context



### Model

In [7]:
def setup_llm_pipeline(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 토크나이저 로드 및 설정
        # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,low_cpu_mem_usage=True)
    
    # # 일부 중요한 레이어는 FP16으로 유지
    # for name, module in model.named_modules():
    #     if "attention" in name or "ffn" in name:  # 중요한 레이어 식별 (예: attention 및 ffn)
    #         module.to(torch.float16)  # 이 부분은 16비트로 유지
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=1024,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
# ghost-x/ghost-8b-beta-1608
# OpenBuddy/openbuddy-llama3.1-8b-v22.3-131k
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.87s/it]


### 점수

In [8]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())
    
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):
    
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    
    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
    
    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)
    
    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

### RUN

In [9]:

def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        line = line.replace('**', '')
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환

def rerun(question,context,answer,llm,num_repeat):
    full_template = "<|begin_of_text|>"
    full_template += """<|start_header_id|>system<|end_header_id|>당신은 이전 답변을 검증하는 챗봇입니다. 질문과 문맥, 이전 답변을 참고해서 지시사항을 따르세요. 지시사항을 따를 때 서론 없이 출력하세요.<|eot_id|>"""
    full_template += f"""<|start_header_id|>user<|end_header_id|>Question: {question} \n\nContexts: {context} \n\nPrevious Answer: {answer} \n\n"""
    full_template += """{input}<|eot_id|>"""
    full_template += """<|start_header_id|>assistant<|end_header_id|>"""
    
    prompt = PromptTemplate(template=full_template)
    chain = (
    {
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
    )
    return chain.invoke("핵심 단어들을 바탕으로, 한 문장으로 요약하세요. 만약 한 문장이라면 그대로 출력하세요.")

def run(faiss,dataset,llm,k=3,verbose=False):
    results = []
    source_path = dataset.iloc[0]['Source_path']
    docs = faiss.similarity_search(
    query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
    filter={"Source_path": source_path},
    k = 99,
    fetch_k = 20000
    )
    buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
    faiss_retriever_mmr = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k})
    # faiss_retriever_sim = buff_faiss.as_retriever(search_kwargs={"k": k})
    # knn_retriever = KNNRetriever.from_documents(docs, embeddings=get_embedding())
    # knn_retriever.k = k
    bm_retriever = OktBM25Retriever.from_documents(docs)
    bm_retriever.k = k

    ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever_mmr,bm_retriever], weight=[0.5,0.5])

    
    for i, row in (dataset.iterrows()):
        if source_path != row['Source_path']:   
            source_path = row['Source_path']
            docs = faiss.similarity_search(
                query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
                filter={"Source_path": source_path},
                k = 99,
                fetch_k = 20000
            )
            buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
            faiss_retriever_mmr = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k})
            # faiss_retriever_sim = buff_faiss.as_retriever(search_kwargs={"k": k})
            # knn_retriever = KNNRetriever.from_documents(docs, embeddings=get_embedding())
            # knn_retriever.k = k
            bm_retriever = OktBM25Retriever.from_documents(docs)
            bm_retriever.k = k

            ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever_mmr,bm_retriever], weight=[0.5,0.5])

            
        full_template = "<|begin_of_text|>"
        full_template += """<|start_header_id|>system<|end_header_id|>
당신은 유용한 금융 정보 QnA 챗봇입니다.
문맥 정보를 활용해서 답변을 작성하세요. 
객관적이고 공식적인 문체를 사용하세요.
서론 없이 핵심 내용을 간결하게 평문으로 작성하세요.<|eot_id|>
"""
        question = row['Question']          
        # full_template += """ """
        contexts = ensemble_retriever.invoke(normalize_string(question))
        contexts = format_docs(contexts)
        
        full_template += "<|start_header_id|>user<|end_header_id|>"
        full_template += "Question: {input}\n\n<|eot_id|>"
        full_template += "Contexts: {contexts}"
        full_template += """<|eot_id|>"""
        full_template += """<|start_header_id|>assistant<|end_header_id>"""
        
        prompt = PromptTemplate(template=full_template, input_variables=["contexts","input"])
        qa_chain = (
        {
            "contexts": RunnablePassthrough(),
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

        answer = qa_chain.invoke({'input':question,'contexts':contexts})
        answer = extract_answer(answer)
        lines = answer.split('\n')
        if  len(lines) > 1 or '|' in answer or ':' in answer:
            previous = answer
            try:
                before = calculate_f1_score(row['Answer'],answer)[2]
            except:
                before = None
            answer = rerun(question=question,
                           context=contexts,
                           answer=answer,
                           llm=llm,
                           num_repeat=1)
        answer = extract_answer(answer)
        results.append({
            "Question": question,
            "Answer": answer,
            "Source": row['Source']
        })
        if verbose:
            print(f"{i}/{len(dataset)}")
            print("Question: ", question, end=" | ")
            print("Context Number |",len(contexts))
            try:
                print(calculate_f1_score(row['Answer'],answer)[2],end=" | ")
            except:
                pass
            print("Answer: ", results[-1]['Answer'])
            try:
                print("Before: ",before," | ",previous)  
                
                previous = None
                before = None
            except:
                pass
            
            try:
                print("REAL Answer: ",row['Answer'])
            except:
                pass
            
            print()
        torch.cuda.empty_cache()
    return results

### 케이폴드

In [10]:
# from sklearn.model_selection import KFold
# import copy
# k_folds = 4
# fold_results = []
# kf = KFold(n_splits=k_folds, shuffle=True, random_state=52)
# for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
#     fold_result = []
#     train_set = train_df.iloc[train_index]
#     val_set = train_df.iloc[val_index]

#     faiss = make_db(val_set)

#     pred = run(faiss,val_set, llm, verbose=True)
#     result = pd.DataFrame()
#     result['pred'] = [result['Answer'] for result in pred]
#     val_set.index = range(len(val_set))
#     result['gt'] = val_set['Answer']
        
#     result = calculate_average_f1_score(result['gt'], result['pred'])
#     print(result)
#     fold_results.append(result)
#     break

### 실전

In [11]:
from save_module import save

faiss= make_db(test_df)

results = run(faiss, test_df, llm, verbose=True)
save(results)

100%|██████████| 9/9 [00:12<00:00,  1.34s/it]


Total number of documents: 228


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\llama\modeling_llama.py:653: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


0/98
Question:  2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요? | Context Number | 821
Answer:  2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다.

1/98
Question:  중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요? | Context Number | 1060
Answer:  중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 창업기반지원 및 개발기술사업화입니다.

2/98
Question:  중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요? | Context Number | 1474
Answer:  중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 중소기업진흥에 관한 법률 제66조, 제67조, 제74조 및 중소기업창업지원법 제35조에 근거하고 있습니다.

3/98
Question:  2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요? | Context Number | 1596
Answer:  2010년에 신규 지원된 혁신창업사업화자금은 재창업자금(실패 경영인에 대한 재기지원)이였습니다.

4/98
Question:  혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요? | Context Number | 1242
Answer:  2020 년에 신규 지원된 혁신창업사업화자금은 미래기술육성자금, 고성장촉진자금입니다.

5/98
Question:  재창업자금이 재도약지원자금으로 이관된 연도는 언제인가요? | Context Number | 1454
Answer:  2015년 재창업자금이 재도약지원자금으로 이관된 연도입니다.

6/98
Question:  창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요? | Context Number | 1790
Answer:  창업기반지원과 신청 대상이 중복인 자금은 일자리창출촉진자금입니다. 이 자금은 2023년 1월에 폐지되었습니다.

7/98


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


...

38/98
Question:  행복주택출자 사업은 어떤 근거로 추진되고 있는가? | Context Number | 1851
Answer:  행복주택출자 사업은 주택도시기금법 제9조제1항가목과 공공주택특별법 제2조1호가목에 근거하고 있습니다.
Before:  None  |  None

39/98
Question:  행복주택출자 사업은 어떤 목적으로 시행되는가? | Context Number | 1966
Answer:  행복주택출자 사업은 국민의 행복주거 실현을 위한 보편적 주거복지 정책의 일환으로 도심 내 다양한 부지를 활용하여 행복주택을 공급하는 목적으로 시행됩니다.
Before:  None  |  None

40/98
Question:  행복주택출자 사업의 주요 수혜자는 누구인가? | Context Number | 1851
Answer:  행복주택 출자 사업의 주요 수혜자는 대학생・사회초년생・신혼부부 등 젊은층(80%), 고령자 및 주거취약계층(20%)입니다.
Before:  None  |  None

41/98
Question:  행복주택출자 사업의 사업비 추이는 어떠한가? | Context Number | 2240
Answer:  행복주택출자 사업의 사업비는 다음과 같습니다.
Before:  None  |  None

42/98
Question:  행복주택출자 사업의 사업시행주체는 누구인가? | Context Number | 2342
Answer:  행복주택출자 사업의 사업시행주체는 한국토지주택공사(LH)와 지자체(지방공사)입니다.
Before:  None  |  None



c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


43/98
Question:  국고보조사업의 보조율은 어떠한 기준에 따라 운용되는가? | Context Number | 2888
Answer:  국고보조사업의 보조율은 「보조금법」과 일부 개별 법령에 근거하여 운영되며, 보조금법에 의해 지방자치단체의 재정여건을 고려하여 기준보조율과 차등보조율을 적용하여 운용됩니다.
Before:  None  |  None

44/98
Question:  프랑스의 재정조정제도에서 최근 강조되는 형평교부금은 어떤 역할을 하는가? | Context Number | 3600
Answer:  수평적 형평교부금의 역할을 강조하는 프랑스의 재정조정제도에서, 최근 강조되는 형평교부금은 중앙정부의 재정적자로 인한 수평적 형평교부금의 역할을 강조하고 있습니다.
Before:  None  |  None

45/98
Question:  지방재정조정제도의 목적은 무엇인가? | Context Number | 3231
Answer:  지방재정조정제도의 목적은 중앙-지방 간 재정 불균형을 완화하고 지방 정부가 본연의 기능을 수행하고 국가 균형발전을 위해 재정을 조정하는 일련의 조치를 의미합니다.
Before:  None  |  None



c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


46/98
Question:  국제적으로 성과중심 재정관리 강화 움직임이 확산된 시기는 언제인가? | Context Number | 2913
Answer:  2007년
Before:  None  |  None

47/98
Question:  한국의 재정사업 성과관리제도는 어떠한 법을 통해 운영되고 있으며, 성과관리 기본계획과 추진계획은 어떻게 의무화되었는가? | Context Number | 2350
Answer:  한국의 재정사업 성과관리제도는 「국가재정법」을 통해 운영되고 있으며, 성과관리 기본계획과 추진계획은 2021년 12월 법 개정을 통해 의무화되었다.
Before:  None  |  None

